**Ref.**
* Statistical Methods for Data Analysis in Particle Physics (Luca Lista) - Chapter 6
* Lyons, L., Gibaut, D., Clifford, P.: How to combine correlated estimates of a single physical quantity. Nucl. Inst. Methods A270, 110–117 (1988)
* A. Valassi, Combining correlated measurements of several different physical quantities. Nucl. Instr. Meth. A 500, 391 (2003) 

# The inputs

Suppose we have,
* $n$ experimental results, denoted as $y_{i}$ = {$y_{1}$, $y_{2}$, $y_{3}$ ...  $y_{n}$}
* Covariance matrix of the measurements $M_{ij} =$ cov($y_{i}$,$y_{j}$) is a $n\times n$ matrix
* $N$ observables, $X_{\alpha}$ = {$X_{1}$,$X_{2}$, $X_{3}$ ... $X_{N}$}

So it's obvious $n = \sum^{N}_{\alpha=1} n_{\alpha} \geq N$

* The link between measurement $y_{i}$ and the observables $X_{\alpha}$ is denoted by a $n \times N$ matrix,

$ 
  U_{i \alpha} = 
\begin{cases}
    1, & \text{if } y_{i} \text{ is a measurement of} X_{\alpha}\\
    0, & \text{if } y_{i} \text{ is a not measurement of} X_{\alpha}
\end{cases}
 $
 
# The desired outputs

* **Observable Estimation:** $\hat{x}_{\alpha}  = \lambda_{\alpha i} y_{i}$ as estimation of the observable $X_{\alpha}$
* **Covariance matrix of measured observables:**  as cov($\hat{x}_{\alpha}$,$\hat{x}_{\beta}$) = $\lambda_{\alpha i} M_{ij} \lambda_{\beta j} $ = $(\lambda M \lambda^{T})_{\alpha \beta}$ 

------------ -----------
* The ref. says $\lambda= (U^{T} M^{-1} U)^{-1} (U^{T} M^{-1})$ , or in index notation $\lambda_{\alpha i} =           \sum^{N}_{\beta = 1} (U^{T} M^{-1} U)_{\alpha \beta}^{-1} (U^{T} M^{-1})_{\beta i}$.

* Puting that in covariance matrix expression we get, 
  cov($\hat{x}_{\alpha}$,$\hat{x}_{\beta}$)= $(U^{T} M^{-1} U)_{\alpha \beta}^{-1}$

-------------------------

* **Decomposition of covariance matrix to statistical and systematics:** Suppose the covariance of measurements can be written as sum of statistical and systematic uncertainty like $M_{ij} = M^{\text{stat}}_{ij} + M^{\text{sys}}_{ij}$. The covariance matrix of observables can also be decomposed as, 


$$\text{ cov}(\hat{x}_{\alpha},\hat{x}_{\beta}) = (\lambda M^{\text{stat} }   \lambda^{T})_{\alpha \beta} +  (\lambda M^{\text{sys} }   \lambda^{T})_{\alpha \beta}$$ 

# Implementation in python script

In [30]:
import numpy as np
from scipy import stats
from sympy import Matrix
def combine_measurement(y, U, M, stat=None, sys=None):
    """
    Combine measurements and give estimated linear observables with covariance matrix
    @param list Measurements  
    @param np.array The link between measurement and observables
    @param np.array Covariance matric
    @param np.array (optional) Statistical component of Covariance matrix 
    @param np.array (optional) Systematics component of Covariance matrix 
    return: 
    """ 
    
    n = U.shape[0]                # to find the number of measurement
    if len(U.shape) > 1:
        N = U.shape[1]
    else:
        N = 1
    Minv = np.linalg.inv(M)       # M^-1
    uT_Minv = np.dot(U.T,Minv)    # UT.M^-1
    uT_Minv_u = np.dot(uT_Minv,U) #UT.M^-1.U

    if len(uT_Minv_u.shape) > 0:
        lambd =  np.dot(np.linalg.inv(uT_Minv_u),uT_Minv) # weight factors calculate
        cov = np.linalg.inv(uT_Minv_u)   
    else:
        lambd =  np.multiply(uT_Minv,1/uT_Minv_u)
        cov = 1/uT_Minv_u
    print('The weights:')
    display(Matrix(lambd))
    x = np.dot(lambd,y)                               # Measure observables 
                                                      # corrlation matrix
    
    if len(x.shape) == 0:        
        print('Measurement:%s+/-%s'%(x,np.sqrt(cov)))
    else:       
        print('Measurement: X = ')
        display(Matrix(x))
        print('\n Covariance matrix: E = ')
        display(Matrix(cov))   
        print('\n sqrt(Covariance matrix): sqrt{E}')
        display(Matrix(np.sqrt(cov)))
        print('\n\n')
    
    # If we need statistical and systematics component individually
    if stat is not None or sys is not None:
        stat_cov = np.dot(np.dot(lambd,stat),lambd.T)
        sys_cov = np.dot(np.dot(lambd,sys),lambd.T)
        
        if len(x.shape) == 0:
            print('Measurement:%s+/-%s +/-%s'%(x,np.sqrt(stat_cov),np.sqrt(sys_cov)))
        else:
            print('\n Estat =  ')
            display(Matrix(stat_cov))   
            print('\n sqrt(Estat) = ')
            display(Matrix(np.sqrt(stat_cov)))
            print('\n\n')
        
        

            print('\n Esys =  ')
            display(Matrix(sys_cov))   
            print('\n sqrt(Esys) = ')
            display(Matrix(np.sqrt(sys_cov)))
            print('\n\n')
            
    yprime = np.subtract(y,np.dot(U,x))
    chi_2 = np.dot(np.dot(yprime.T,Minv),yprime)
    p = stats.chi2.pdf(chi_2 , n-N)
    print('Minimum chisqauare for the combination is %s (d.o.f = %s)  with p value %s'%(chi_2,n-N, p))

```python
import numpy as np
from scipy import stats
from sympy import Matrix
def combine_measurement(y, U, M, stat=None, sys=None):
    """
    Combine measurements and give estimated linear observables with covariance matrix
    @param list Measurements  
    @param np.array The link between measurement and observables
    @param np.array Covariance matric
    @param np.array (optional) Statistical component of Covariance matrix 
    @param np.array (optional) Systematics component of Covariance matrix 
    return: 
    """ 
    
    n = U.shape[0]                # to find the number of measurement
    if len(U.shape) > 1:
        N = U.shape[1]
    else:
        N = 1
    Minv = np.linalg.inv(M)       # M^-1
    uT_Minv = np.dot(U.T,Minv)    # UT.M^-1
    uT_Minv_u = np.dot(uT_Minv,U) #UT.M^-1.U

    if len(uT_Minv_u.shape) > 0:
        lambd =  np.dot(np.linalg.inv(uT_Minv_u),uT_Minv) # weight factors calculate
        cov = np.linalg.inv(uT_Minv_u)   
    else:
        lambd =  np.multiply(uT_Minv,1/uT_Minv_u)
        cov = 1/uT_Minv_u
    print('The weights:')
    display(Matrix(lambd))
    x = np.dot(lambd,y)                               # Measure observables 
                                                      # corrlation matrix
    
    if len(x.shape) == 0:        
        print('Measurement:%s+/-%s'%(x,np.sqrt(cov)))
    else:       
        print('Measurement: X = ')
        display(Matrix(x))
        print('\n Covariance matrix: E = ')
        display(Matrix(cov))   
        print('\n sqrt(Covariance matrix): sqrt{E}')
        display(Matrix(np.sqrt(cov)))
        print('\n\n')
    
    # If we need statistical and systematics component individually
    if stat is not None or sys is not None:
        stat_cov = np.dot(np.dot(lambd,stat),lambd.T)
        sys_cov = np.dot(np.dot(lambd,sys),lambd.T)
        
        if len(x.shape) == 0:
            print('Measurement:%s+/-%s +/-%s'%(x,np.sqrt(stat_cov),np.sqrt(sys_cov)))
        else:
            print('\n Estat =  ')
            display(Matrix(stat_cov))   
            print('\n sqrt(Estat) = ')
            display(Matrix(np.sqrt(stat_cov)))
            print('\n\n')
        
        

            print('\n Esys =  ')
            display(Matrix(sys_cov))   
            print('\n sqrt(Esys) = ')
            display(Matrix(np.sqrt(sys_cov)))
            print('\n\n')
            
    yprime = np.subtract(y,np.dot(U,x))
    chi_2 = np.dot(np.dot(yprime.T,Minv),yprime)
    p = stats.chi2.pdf(chi_2 , n-N)
    print('Minimum chisqauare for the combination is %s (d.o.f = %s)  with p value %s'%(chi_2,n-N, p))```

# Example


Measurement of Branching fraction in $e$ and $\tau$ decay channel in two different experiment.

* $\mathcal{\hat{B}}^{e}_{A} = (10.50  \pm 1)\%$
* $\mathcal{\hat{B}}^{e}_{B} = (13.50  \pm 3)\%$
* $\mathcal{\hat{B}}^{\tau}_{A} = (9.50  \pm 3)\%$
* $\mathcal{\hat{B}}^{\tau}_{B} = (14.00  \pm 3)\%$


So using the same notation, four meaurements, $B_{i}$ = {$\mathcal{\hat{B}}^{e}_{A}$,$\mathcal{\hat{B}}^{e}_{B}$, $\mathcal{\hat{B}}^{\tau}_{A}$, $\mathcal{\hat{B}}^{\tau}_{B}$} and define two observables ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$}. 

In [31]:
U42 = np.array([[1,0],[1,0],[0,1],[0,1]])
print('The link matrix U = ')
display(Matrix(U42))

The link matrix U = 


Matrix([
[1, 0],
[1, 0],
[0, 1],
[0, 1]])

## No correlations

In [32]:
y = 1e-2*np.array([10.50,13.50,9.50,14.00])
print('So the measurement matrix will be: y=')
display(Matrix(y))

M =  1e-4*np.array([[1,0, 0, 0], [0, 9,0,0],[0,0,9,0],[0,0,0,9]])
print('and its covariance matrix will be: M=')
display(Matrix(M))

So the measurement matrix will be: y=


Matrix([
[0.105],
[0.135],
[0.095],
[ 0.14]])

and its covariance matrix will be: M=


Matrix([
[0.0001,    0.0,    0.0,    0.0],
[   0.0, 0.0009,    0.0,    0.0],
[   0.0,    0.0, 0.0009,    0.0],
[   0.0,    0.0,    0.0, 0.0009]])

***

* Now lets use the script to get the output executing, 

`combine_measurement(y, U, M)`

In [33]:
combine_measurement(y, U42, M, stat=None, sys=None)

The weights:


Matrix([
[0.9, 0.1, 0.0, 0.0],
[0.0, 0.0, 0.5, 0.5]])

Measurement: X = 


Matrix([
[ 0.108],
[0.1175]])


 Covariance matrix: E = 


Matrix([
[9.0e-5,     0.0],
[   0.0, 0.00045]])


 sqrt(Covariance matrix): sqrt{E}


Matrix([
[0.00948683298050514,                0.0],
[                0.0, 0.0212132034355964]])




Minimum chisqauare for the combination is 2.0250000000000012 (d.o.f = 2)  with p value 0.1816547846795055


***
***

Now suppose assuming LFU we are interested in one observables ${B_{\alpha}}$ = {${B^{\ell}}$}.


In [34]:
U41 = np.array([1,1,1,1])
print('The link matrix U = ')
display(Matrix(U41))

The link matrix U = 


Matrix([
[1],
[1],
[1],
[1]])

The script gives us, 

In [35]:
combine_measurement(y, U41, M, stat=None, sys=None)

The weights:


Matrix([
[              0.75],
[0.0833333333333333],
[0.0833333333333333],
[0.0833333333333333]])

Measurement:0.10958333333333332+/-0.008660254037844387
Minimum chisqauare for the combination is 2.192129629629631 (d.o.f = 3)  with p value 0.19739142442639945


## Correlations between measuremens of same observables

Let's assume that a 15% correlation exists between measrements performed by A and B for the same errors given before. So correlatoion will be $\rho \sigma_{1}\sigma_{1} = 0.15 \times 1 \times 3 \times 10^{-4} =0.45 \times 10^{-4}$

In [36]:
M =  1e-4*np.array([[1,0.45, 0, 0], [0.45, 9,0,0],[0,0,9,0],[0,0,0,9]])
print('The covariance matrix will be: M=')
display(Matrix(M))

The covariance matrix will be: M=


Matrix([
[0.0001, 4.5e-5,    0.0,    0.0],
[4.5e-5, 0.0009,    0.0,    0.0],
[   0.0,    0.0, 0.0009,    0.0],
[   0.0,    0.0,    0.0, 0.0009]])

* Measurement of  ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$} gives,

In [37]:
combine_measurement(y, U42, M, stat=None, sys=None)

The weights:


Matrix([
[0.93956043956044, 0.0604395604395604, 0.0, 0.0],
[             0.0,                0.0, 0.5, 0.5]])

Measurement: X = 


Matrix([
[0.106813186813187],
[           0.1175]])


 Covariance matrix: E = 


Matrix([
[9.66758241758242e-5,     0.0],
[                0.0, 0.00045]])


 sqrt(Covariance matrix): sqrt{E}


Matrix([
[0.00983238649442871,                0.0],
[                0.0, 0.0212132034355964]])




Minimum chisqauare for the combination is 2.1140109890109904 (d.o.f = 2)  with p value 0.17374741452746106


* Measurement of  ${B_{\alpha}} =$ {$B^{\ell}$} gives,

In [38]:
combine_measurement(y, U41, M, stat=None, sys=None)

The weights:


Matrix([
[ 0.773405698778833],
[0.0497512437810945],
[0.0884215287200362],
[0.0884215287200362]])

Measurement:0.10870307553143374+/-0.008920727316089902
Minimum chisqauare for the combination is 2.3229245188200434 (d.o.f = 3)  with p value 0.19033162900540035


## Correlation between measurements of different observables

### Positive correlation

Now suppose +99.5% correlation exists between measurements of $B^{e}$ and $B^{\tau}$ performed by B, while the results of A and  B are uncorrelated. In the same experiment there are se

In [39]:
M =  1e-4*np.array([[1,0, 0, 0], [0, 9,0,8.96],[0,0,9,0],[0,8.96,0,9]])
print('The covariance matrix will be: M=')
display(Matrix(M))

The covariance matrix will be: M=


Matrix([
[0.0001,      0.0,    0.0,      0.0],
[   0.0,   0.0009,    0.0, 0.000896],
[   0.0,      0.0, 0.0009,      0.0],
[   0.0, 0.000896,    0.0,   0.0009]])

* This leads to measurement of  ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$} as,

In [40]:
combine_measurement(y, U42, M, stat=None, sys=None)

The weights:


Matrix([
[0.819491688595084,  0.180508311404916, 0.0898530261215584, -0.089853026121558],
[0.808677235094026, -0.808677235094026, 0.0974584429754188,  0.902541557024581]])

Measurement: X = 


Matrix([
[0.106371863166678],
[0.111354053013285]])


 Covariance matrix: E = 


Matrix([
[8.19491688595084e-5, 8.08677235094026e-5],
[8.08677235094026e-5, 8.77125986778769e-5]])


 sqrt(Covariance matrix): sqrt{E}


Matrix([
[0.00905257802283463, 0.00899264830344224],
[0.00899264830344224,  0.0093655004499427]])




Minimum chisqauare for the combination is 1.22926160066748 (d.o.f = 2)  with p value 0.2704202682994562


* This leads to measurement of  ${B_{\alpha}} =$ {$B^{\ell}$} as,

In [41]:
combine_measurement(y, U41, M, stat=None, sys=None)

The weights:


Matrix([
[ 0.818016194331984],
[0.0455465587044532],
[0.0908906882591094],
[0.0455465587044532]])

Measurement:0.10705161943319838+/-0.009044424770719166
Minimum chisqauare for the combination is 4.360880566801648 (d.o.f = 3)  with p value 0.09413345065568042


### Negetive correlation

In the example above let the correlation between  $\mathcal{\hat{B}}^{e}_{A}$ and $\mathcal{\hat{B}}^{e}_{A}$ be equal to -99.5%. 

**Negetive correlation can be understood in this case as mis-identification of  electron as tau or vice virsa.**


In [42]:
M =  1e-4*np.array([[1,0, 0, 0], [0, 9,0,-8.96],[0,0,9,0],[0,-8.96,0,9]])
print('The covariance matrix will be: M=')
display(Matrix(M))

The covariance matrix will be: M=


Matrix([
[0.0001,       0.0,    0.0,       0.0],
[   0.0,    0.0009,    0.0, -0.000896],
[   0.0,       0.0, 0.0009,       0.0],
[   0.0, -0.000896,    0.0,    0.0009]])

* This leads to measurement of  ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$} as,

In [43]:
combine_measurement(y, U42, M, stat=None, sys=None)

The weights:


Matrix([
[ 0.819491688595084, 0.180508311404916, -0.0898530261215584, 0.089853026121558],
[-0.808677235094026, 0.808677235094026,  0.0974584429754188, 0.902541557024581]])

Measurement: X = 


Matrix([
[0.114458635517618],
[0.159874687118927]])


 Covariance matrix: E = 


Matrix([
[ 8.19491688595084e-5, -8.08677235094026e-5],
[-8.08677235094026e-5,  8.77125986778769e-5]])


 sqrt(Covariance matrix): sqrt{E}


/home/soumen/source/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in sqrt


Matrix([
[0.00905257802283463,                nan],
[                nan, 0.0093655004499427]])




Minimum chisqauare for the combination is 6.081325011231629 (d.o.f = 2)  with p value 0.02390160455332634


* This leads to measurement of  ${B_{\alpha}} =$ {$B^{\ell}$} as,

In [44]:
combine_measurement(y, U41, M, stat=None, sys=None)

The weights:


Matrix([
[ 0.0195652173913041],
[  0.489130434782609],
[0.00217391304347824],
[  0.489130434782609]])

Measurement:0.13677173913043478+/-0.0013987572123604632
Minimum chisqauare for the combination is 12.305329476130543 (d.o.f = 3)  with p value 0.002977750761029625


### Breakdown of error contributions

In [45]:
STAT = 1e-4*np.array([[1,0, 0, 0], [0, 0.04,0,0],[0,0,9,0],[0,0,0,0.04]])
SYS1 =  1e-4*np.array([[0,0, 0, 0], [0, 8.96,0,8.96],[0,0,0,0],[0,8.96,0,8.96]])
SYS2 =  1e-4*np.array([[0,0, 0, 0], [0, 8.96,0,0],[0,0,0,0],[0,0,0,8.96]])
SYS3 =  1e-4*np.array([[0,0, 0, 0], [0, 8.96,0,-8.96],[0,0,0,0],[0,-8.96,0,8.96]])

SYS0 =  1e-4*np.array([[0,0, 0, 0], [0, 0,0,0],[0,0,0,0],[0,0,0,0]])

In [46]:
print('The statictical compoments of covariance matrix is :  MStat')
display(Matrix(STAT))

The statictical compoments of covariance matrix is :  MStat


Matrix([
[0.0001,    0.0,    0.0,    0.0],
[   0.0, 4.0e-6,    0.0,    0.0],
[   0.0,    0.0, 0.0009,    0.0],
[   0.0,    0.0,    0.0, 4.0e-6]])

* For  $\mathcal{\hat{B}}^{e}_{B}$ and $\mathcal{\hat{B}}^{\tau}_{B}$ +100% correlated scenario
   *  ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$} 
   *  ${B_{\alpha}} =$ {$B^{\ell}$} 
   

In [47]:
print('The systematic compoments of covariance matrix is :  MSys')
display(Matrix(SYS1))
M = STAT+SYS1
combine_measurement(y, U42, M, stat=STAT, sys=SYS1)

The systematic compoments of covariance matrix is :  MSys


Matrix([
[0.0,      0.0, 0.0,      0.0],
[0.0, 0.000896, 0.0, 0.000896],
[0.0,      0.0, 0.0,      0.0],
[0.0, 0.000896, 0.0, 0.000896]])

The weights:


Matrix([
[0.819491688595084,  0.180508311404916, 0.0898530261215584, -0.089853026121558],
[0.808677235094026, -0.808677235094026, 0.0974584429754188,  0.902541557024581]])

Measurement: X = 


...





 Esys =  


Matrix([
[7.36366915182715e-6, 7.62433000675895e-6],
[7.62433000675896e-6, 7.89421779460871e-6]])


 sqrt(Esys) = 


Matrix([
[0.00271360814264461, 0.00276121893495589],
[0.00276121893495589, 0.00280966506804792]])




Minimum chisqauare for the combination is 1.22926160066748 (d.o.f = 2)  with p value 0.2704202682994562


In [48]:
combine_measurement(y, U41, M, stat=STAT, sys=SYS1)

The weights:


Matrix([
[ 0.818016194331984],
[0.0455465587044532],
[0.0908906882591094],
[0.0455465587044532]])

Measurement:0.10705161943319838+/-0.009044424770719166
Measurement:0.10705161943319838+/-0.008623610080587478 +/-0.002726713885098403
Minimum chisqauare for the combination is 4.360880566801648 (d.o.f = 3)  with p value 0.09413345065568042


* For  $\mathcal{\hat{B}}^{e}_{B}$ and $\mathcal{\hat{B}}^{\tau}_{B}$ uncorrelated scenario
   *  ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$} 
   *  ${B_{\alpha}} =$ {$B^{\ell}$} 
   

In [49]:
print('The systematic compoments of covariance matrix is :  MSys')
display(Matrix(SYS2))
M = STAT+SYS2
combine_measurement(y, U42, M, stat=STAT, sys=SYS2)

The systematic compoments of covariance matrix is :  MSys


Matrix([
[0.0,      0.0, 0.0,      0.0],
[0.0, 0.000896, 0.0,      0.0],
[0.0,      0.0, 0.0,      0.0],
[0.0,      0.0, 0.0, 0.000896]])

The weights:


Matrix([
[0.9, 0.1, 0.0, 0.0],
[0.0, 0.0, 0.5, 0.5]])

Measurement: X = 


...





 Esys =  


Matrix([
[8.96e-6,      0.0],
[    0.0, 0.000224]])


 sqrt(Esys) = 


Matrix([
[0.00299332590941915,                0.0],
[                0.0, 0.0149666295470958]])




Minimum chisqauare for the combination is 2.0250000000000012 (d.o.f = 2)  with p value 0.1816547846795055


In [50]:
combine_measurement(y, U41, M, stat=STAT, sys=SYS2)

The weights:


Matrix([
[              0.75],
[0.0833333333333333],
[0.0833333333333333],
[0.0833333333333333]])

Measurement:0.10958333333333332+/-0.008660254037844387
Measurement:0.10958333333333332+/-0.007909207011803114 +/-0.0035276684147527875
Minimum chisqauare for the combination is 2.192129629629631 (d.o.f = 3)  with p value 0.19739142442639945


* For  $\mathcal{\hat{B}}^{e}_{B}$ and $\mathcal{\hat{B}}^{\tau}_{B}$ -100% scenario
   *  ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$} 
   *  ${B_{\alpha}} =$ {$B^{\ell}$} 
   

In [51]:
print('The systematic compoments of covariance matrix is :  MSys')
display(Matrix(SYS3))
M = STAT+SYS3
combine_measurement(y, U42, M, stat=STAT, sys=SYS3)

The systematic compoments of covariance matrix is :  MSys


Matrix([
[0.0,       0.0, 0.0,       0.0],
[0.0,  0.000896, 0.0, -0.000896],
[0.0,       0.0, 0.0,       0.0],
[0.0, -0.000896, 0.0,  0.000896]])

The weights:


Matrix([
[ 0.819491688595084, 0.180508311404916, -0.0898530261215584, 0.089853026121558],
[-0.808677235094026, 0.808677235094026,  0.0974584429754188, 0.902541557024581]])

Measurement: X = 


...

Matrix([
[ 7.36366915182715e-6, -7.62433000675895e-6],
[-7.62433000675896e-6,  7.89421779460871e-6]])


 sqrt(Esys) = 


/home/soumen/source/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in sqrt


Matrix([
[0.00271360814264461,                 nan],
[                nan, 0.00280966506804792]])




Minimum chisqauare for the combination is 6.081325011231629 (d.o.f = 2)  with p value 0.02390160455332634


In [52]:
combine_measurement(y, U41, M, stat=STAT, sys=SYS3)

The weights:


Matrix([
[ 0.0195652173913041],
[  0.489130434782609],
[0.00217391304347824],
[  0.489130434782609]])

Measurement:0.13677173913043478+/-0.0013987572123604632
Measurement:0.13677173913043478+/-0.0013987572123604708 +/-0.0
Minimum chisqauare for the combination is 12.305329476130543 (d.o.f = 3)  with p value 0.002977750761029625


* Under the assumption systematic uncertainty is zero,
   *  ${B_{\alpha}} =$ {$B^{e}, B^{\tau}$} 
   *  ${B_{\alpha}} =$ {$B^{\ell}$} 


In [53]:
STAT0 = 1e-4*np.array([[1,0, 0, 0], [0, 0.0441,0,0],[0,0,9,0],[0,0,0,0.0441]])
print('The statistical compoments of covariance matrix is :  MStat')
display(Matrix(STAT0))
M = STAT0+SYS0
combine_measurement(y, U42, M, stat=STAT0, sys=SYS0)

The statistical compoments of covariance matrix is :  MStat


Matrix([
[0.0001,     0.0,    0.0,     0.0],
[   0.0, 4.41e-6,    0.0,     0.0],
[   0.0,     0.0, 0.0009,     0.0],
[   0.0,     0.0,    0.0, 4.41e-6]])

The weights:


Matrix([
[0.0422373335887367, 0.957762666411263,                 0.0,               0.0],
[               0.0,               0.0, 0.00487610707533088, 0.995123892924669]])

Measurement: X = 


...





 Esys =  


Matrix([
[0.0, 0.0],
[0.0, 0.0]])


 sqrt(Esys) = 


Matrix([
[0.0, 0.0],
[0.0, 0.0]])




Minimum chisqauare for the combination is 10.858892756781882 (d.o.f = 2)  with p value 0.0021927615255231438


In [54]:
combine_measurement(y, U41, M, stat=STAT0, sys=SYS0)

The weights:


Matrix([
[ 0.0215226939970717],
[  0.488042947779405],
[0.00239141044411908],
[  0.488042947779405]])

Measurement:0.13669887750122012+/-0.0014670614846376325
Measurement:0.13669887750122012+/-0.0014670614846376325 +/-0.0
Minimum chisqauare for the combination is 15.105715967857792 (d.o.f = 3)  with p value 0.0008134224940035461


# Negetive weights and interpretation

You might have been noticed some of the weights are negetive. This cannot be interpreted properly. We have to take special treatment for those cases. The references below shows proper guide map.

* Valassi, A., Chierici, R.: Information and treatment of unknown correlations in the combination of measurements using the BLUE method. Eur. Phys. J. C 74, 2717 (2014)
* Lista, L.: The bias of the unbiased estimator: a study of the iterative application of the BLUE method. Nucl. Inst. Methods A764, 82–93 (2014) and corr. ibid. A773, 87–96 (2015)

Moreover there are efforts to include these algorithm to ROOT framework. [Here](https://blue.hepforge.org/) is the link of th webpage. 